In [1]:
% load_ext autoreload
% autoreload 2

In [2]:
#! git clone https://github.com/stankevich-mipt/text_to_image_with_transformer.git

In [3]:
! nvidia-smi

Sun Jun 13 23:03:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#! pip install subword-nmt
#! pip install nltk
#! pip install torchtext
#! pip install einops

In [5]:
#!pip install pydrive                             
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth                    
#from oauth2client.client import GoogleCredentials

#auth.authenticate_user()                         
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

#your_module = drive.CreateFile({"id": "1-1zRjcpx_UBFGCWD62r5O9POl5krUYOo"})   
#your_module.GetContentFile("flowers_.hdf5")

In [6]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), 'text_to_image_with_transformer'))

In [7]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformer.dataset import Text2ImageDataset

train_dataset = Text2ImageDataset(
    'flowers_.hdf5',
    max_text_length=64,
    split=0
)

In [8]:
from transformer.transformer import Transformer
from torch.optim import AdamW

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = Transformer(
    train_dataset.image_tokens, train_dataset.text_tokens, 
    train_dataset.max_text_length, 258, device, num_layers=6
)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [11]:
from transformer.transformer_trainer import Trainer

trainer = Trainer(
    model, optimizer, device, 
    train_dataset, 
    snapshot_path='/content/drive/MyDrive/transformer_flowers/' 
)
trainer.train(from_zero=True)